# Confluence Module SLURM Script Generator

In [34]:
import os
import subprocess as sp
mnt_dir = '/nas/cee-water/cjgleason/travis/data/france_test/empty_mnt' # Downloaded using: gdown 1xRltFZ1gyP_nvwHMJW-rIgClzXx8CSLC
sif_dir = '/nas/cee-water/cjgleason/travis/data/france_test' # This should be an existing repository
report_dir = '/nas/cee-water/cjgleason/travis/data/france_test/report' # This should be an existing repository
docker_username = 'travissimmons' # Use travissimmons for operational confluence modules, use your account for custom modules
submission_prefix = '#SBATCH'

module_to_run = 'prediagnostics' # Chose what module to run using the below command dict, they are listed in order. eg: run expanded_setfinder first

# Customize this dictionary as you see fit to generate a SLURM submission script for your institution
job_details = {
    'partition': 'cpu-preempt',
    'cpus-per-task':'1',
    'mem':'2G',
    'time': '00:10:00',
    'job-name': 'cfl_prediag',
}

# This is a dictionary of all of the Confluence module run commands translated to singularity run commands.
# You should not have to change anything here.
# If you find yourself changing things here, please reach out to me.
command_dict = {
    'expanded_setfinder': 'singularity run --bind '+ f'{mnt_dir}/input:/data ' + os.path.join(sif_dir, 'setfinder.simg') + ' -r reaches_of_interest.json -c continent.json -e -s 16 -o /data -n /data -a MetroMan HiVDI SIC NeoBAM -i ${SLURM_ARRAY_TASK_ID}',
    'expanded_combine_data': 'singularity run --bind ' +f'{mnt_dir}/input:/data ' +os.path.join(sif_dir, 'combine_data.simg') + ' -d /data  -e -s 16',
    'input': 'singularity run --bind ' + f'{mnt_dir}/input:/mnt/data ' + os.path.join(sif_dir, 'input.simg') + ' -r /mnt/data/expanded_reaches_of_interest.json' + ' -i ${SLURM_ARRAY_TASK_ID}',
    'non_expanded_setfinder': 'singularity run --bind '+ f'{mnt_dir}/input:/data ' + os.path.join(sif_dir, 'setfinder.simg') + ' -c continent.json -s 16 -o /data -n /data -a MetroMan HiVDI SIC NeoBAM -i ${SLURM_ARRAY_TASK_ID}',
    'non_expanded_combine_data': 'singularity run --bind ' +f'{mnt_dir}/input:/data ' +os.path.join(sif_dir, 'combine_data.simg') + ' -d /data -s 16',
    'prediagnostics': 'singularity run --bind ' +f'{mnt_dir}/input:/mnt/data/input,{mnt_dir}/diagnostics/prediagnostics:/mnt/data/output ' +  os.path.join(sif_dir,'prediagnostics.simg') + ' -i ${SLURM_ARRAY_TASK_ID} -r reaches.json',
    'unconstrained_priors': f'singularity run -c --writable-tmpfs --bind {mnt_dir}/input:/mnt/data {os.path.join(sif_dir, "priors.simg")} ' + ' -i ${SLURM_ARRAY_TASK_ID} -r unconstrained -p usgs riggs -g -s local',
    'metroman': f'singularity run --env AWS_BATCH_JOB_ID=”foo” --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/metroman:/mnt/data/output ' +  os.path.join(sif_dir,"metroman.simg") + ' -i ${SLURM_ARRAY_TASK_ID} -r metrosets.json -s local -v',
    'metroman_consolidation': f'singularity run --bind {mnt_dir}/input:/mnt/input,{mnt_dir}/flpe:/mnt/flpe ' +  os.path.join(sif_dir,'metroman_consolidation.simg') + ' -i ${SLURM_ARRAY_TASK_ID}',
    'unconstrained_momma': f'singularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/momma:/mnt/data/output ' + os.path.join(sif_dir, 'momma.simg') + ' -r reaches.json -m 3 -i ${SLURM_ARRAY_TASK_ID}',
    'neobam': f'singularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/geobam:/mnt/data/output '+ os.path.join(sif_dir, 'neobam.simg') + ' -r reaches.json -i ${SLURM_ARRAY_TASK_ID}',
    'sad': f'singularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/sad:/mnt/data/output '+ os.path.join(sif_dir, 'sad.simg') + ' --reachfile reaches.json --index ${SLURM_ARRAY_TASK_ID}',
    'moi': f'singularity run --env AWS_BATCH_JOB_ID="foo" --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe:/mnt/data/flpe,{mnt_dir}/moi:/mnt/data/output ' + os.path.join(sif_dir, 'moi.simg') + ' -j basin.json -v -b unconstrained -s local -i ${SLURM_ARRAY_TASK_ID}',
    'unconstrained_offline': f'singularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe:/mnt/data/flpe,{mnt_dir}/moi:/mnt/data/moi,{mnt_dir}/offline:/mnt/data/output ' + os.path.join(sif_dir, 'offline.simg') + ' unconstrained timeseries integrator reaches.json ${SLURM_ARRAY_TASK_ID}',
    'validation': f'singularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe:/mnt/data/flpe,{mnt_dir}/moi:/mnt/data/moi,{mnt_dir}/offline:/mnt/data/offline,{mnt_dir}/validation:/mnt/data/output ' + os.path.join(sif_dir, 'validation.simg') + ' reaches.json unconstrained ${SLURM_ARRAY_TASK_ID}',
    'output': f'singularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe:/mnt/data/flpe,{mnt_dir}/moi:/mnt/data/moi,{mnt_dir}/diagnostics:/mnt/data/diagnostics,{mnt_dir}/offline:/mnt/data/offline,{mnt_dir}/validation:/mnt/data/validation,{mnt_dir}/output:/mnt/data/output ' + os.path.join(sif_dir, 'output.simg') + ' -s local -j /app/metadata/metadata.json -m input priors prediagnostics momma neobam metroman sic4dvar sad moi offline validation swot -i ${SLURM_ARRAY_TASK_ID}'
}

job_details['run_command'] = command_dict[module_to_run]
job_details['module_name'] =  module_to_run
job_details['docker_username']= docker_username
job_details['submission_prefix']= submission_prefix

In [35]:
def create_slurm_script(job_details=job_details, build_image=False, sif_dir='foo'):
    submission_prefix = job_details['submission_prefix']
    if build_image:
        module_name = job_details['module_name']
        image_name = module_name.replace('expanded_', '').replace('non_', '').replace('unconstrained_','').replace('constrained_','')
        sp.run(['singularity', 'build', '-F',os.path.join(sif_dir,image_name + '.simg'), f'docker://{job_details['docker_username']}/{image_name}'])
        
    # sh_dict = construct_sh_dict(job_details=job_details)
    file = open(f'{module_to_run}.sh','w')
    file.write('#!/bin/bash \n')
    file.write(f'{submission_prefix} -o {os.path.join(report_dir, f"{module_to_run}.%a.out")}' + ' \n')

    for item in job_details:
        if item not in  ['run_command', 'module_name', 'docker_username', 'submission_prefix']:
            file.write(f'{submission_prefix} --{item}={job_details[item]} \n')
    file.write(job_details["run_command"])
    file.close()

INFO:    Starting build...
Copying blob sha256:744b4807d9e87e2a9a46a5f1ace4e204411ff097ea3b619cfe0117c9800245cc
Copying blob sha256:5504daec233685bd41592a251e44bd91cf2fcc3b1bf891e7c3a72627a5462730
Copying blob sha256:7a0faad6d294c3a9fdd6ba76dab70d06a10e6241ae0a82a40b79f23785f8d7fe
Copying blob sha256:de44b265507ae44b212defcb50694d666f136b35c1090d9709068bc861bb2d64
Copying blob sha256:401811458ff6782f0a2172f9cdc7b311bbd51032dda0ce5601b76f999e2d64e0
Copying blob sha256:b0d3d340805ee51b2b2b123e4e15e589cac12ddb81d3551a1f1dc8210c2297fd
Copying blob sha256:c6cf0bcec8e188d39ded0332efc939d1912b59c71fed8bb7e9ac7552a1562fdb
Copying config sha256:ff95a7b3fd77991c873c551ebe71f89c6622d4b80b8942f2894bbc143f4d3088
Writing manifest to image destination
2025/02/05 04:57:35  info unpack layer: sha256:de44b265507ae44b212defcb50694d666f136b35c1090d9709068bc861bb2d64
2025/02/05 04:57:36  info unpack layer: sha256:744b4807d9e87e2a9a46a5f1ace4e204411ff097ea3b619cfe0117c9800245cc
2025/02/05 04:57:41  info unp

In [ ]:

# Run this to create the SLURM script. Set the build image to True to build the image the first time, or re-build it.
# Re-build the image if you customize it, or if it has been a while, as my dockerhub may be updated.
create_slurm_script( job_details=job_details, build_image= True, sif_dir = sif_dir)

In [1]:
# After running this notebook, there will be a {module name}.sh file generated in the same directory.
# You can either add in an array and submit the job using sbatch or you can fill out the top of the cfl_wrapper.sh and have it submit jobs for you.
# using the cfl_wrapper.sh is highly recommended if you are submitting a number of jobs larger than your HPC allows. I use it in all cases though.